In [2]:
import numpy as np
import pandas as pd

In [ ]:
# TAN算法,树增强型贝叶斯算法（Tree Augmented Naive Bayes）
class Tan(NBayes):
    
    def __init__(self):
        super(Tan, self).__init__()
        self.CMI_dict = dict()                      #条件互信息字典
        self.f_relationship = dict()                #特征依赖关系{子特征：父特征（依赖属性）}
   
    # 计算分类任务的条件互信息
    def __CMI_classfic(self, xi, xj, y):
        yset = np.unique(y)
        y_count = y.size
        cmi = 0
        for yi in yset:
            yi_idx = np.nonzero(np.array(y==yi))[0]
            yi_count = yi_idx.size
            yi_proba = yi_count/y_count
            arr0 = xi[yi_idx]
            arr1 = xj[yi_idx]
            mi = self.__calDiscreteMutualInformation(arr0, arr1)
            cmi += mi * yi_proba
        return cmi
            
    # 计算两个离散特征的互信息
    def __calDiscreteMutualInformation(self, arr0, arr1):
        # sklearn中现成的
        #from sklearn import metrics
        #metrics.mutual_info_score(arr0, arr1)
        # 自己写的
        mi = 0
        if len(arr0) != len(arr1):
            raise ValueError("arr0's length should be qeual to arr1's length!")
        else:
            n_samples = len(arr0)
            # 数组一的分布统计
            setX0 = np.unique(arr0)
            bincountX0 = {}
            for xi in setX0:
                bincountX0[xi] = sum(arr0 == xi)
            # 数组二的分布统计
            setX1 = np.unique(arr1)
            bincountX1 = {}
            for xj in setX1:
                bincountX1[xj] = sum(arr1 == xj)
            # 数组一和数组二的联合分布统计
            for i in setX0:
                px0 = bincountX0[i]/n_samples
                for j in setX1:
                    px1 = bincountX1[j]/n_samples
                    px0x1 = sum(np.equal(arr0, i)&np.equal(arr1, j))/n_samples
                    if px0x1 == 0:
                        continue
                    mi += px0x1*np.log(px0x1/(px0*px1))
        return mi
    
    # 计算特征之间的条件互信息，并生成字典，目前只生成离散特征之间的依赖关系
    def get_cmidict(self, X, y, columnsMark):
        n_samples, n_features = X.shape
        CMI_dict = dict()
        featureIdx = np.nonzero(np.array(columnsMark)==0)[0]
        for idx, i in enumerate(featureIdx[:-1]):
            for j in featureIdx[idx+1:]:
                CMI_dict[(i, j)] = self.__CMI_classfic(X.iloc[:,i], X.iloc[:,j], y)
        return CMI_dict
    
    # 获取特征的依赖关系
    def get_relationship(self, features:list, weight:list):
        # 获取最大次数的顶点
        point_count = dict()
        for p0, p1 in features:
            if p0 not in point_count.keys():
                point_count[p0] = 1
            else:
                point_count[p0] += 1
            if p1 not in point_count.keys():
                point_count[p1] = 1
            else:
                point_count[p1] += 1
        pointcntList = sorted(point_count.items(), key=lambda x: x[1], reverse=True)
        maxcntFeature = pointcntList[0][0]
        # 遍历特征，保存依赖关系
        feature_relationship = dict()
        feature_epoch = [maxcntFeature]
        features_index = []
        while len(features_index) < len(features):
            for idx, feature_pair in enumerate(features):
                if idx in features_index:
                    continue
                if feature_pair[0] in feature_epoch:
                    feature_relationship[feature_pair[1]] = feature_pair[0]
                    feature_epoch.append(feature_pair[1])
                    features_index.append(idx)
                elif feature_pair[1] in feature_epoch:
                    feature_relationship[feature_pair[0]] = feature_pair[1]
                    feature_epoch.append(feature_pair[0])
                    features_index.append(idx)
                else:
                    feature_relationship[feature_pair[1]] = feature_pair[0]
                    feature_epoch.append(feature_pair[1])
                    features_index.append(idx)
        return feature_relationship
    
    # TAN算法训练
    def TanTrain(self, X, y, columnsMark):
        # 1 根据最大带权生成树找到每个特征的依赖属性，目前只在离散变量之间做了父属性
        ## 1.1 计算互信息字典
        self.CMI_dict = self.get_cmidict(X, y, columnsMark)
        
        ## 1.2 生成最大带权树
        points = list(set([i[0] for i in self.CMI_dict.keys()]+[i[1] for i in self.CMI_dict.keys()]))
        self.MaxstClass = MST('max', 'Kruskal')
        self.Maxspanningtree = self.MaxstClass.fit_transform(points, self.CMI_dict)
        
        ## 1.3自定义顶点，使之有向，构建出依赖关系
        self.f_relationship = self.get_relationship([f for f, w in self.Maxspanningtree], [w for f, w in self.Maxspanningtree])
        
        # 2 训练贝叶斯的联合概率、条件概率
        ## 2.1 初始化变量，样本数量、特征数量、标签类别、
        ##     类别的先验联合概率、联合概率的的条件概率
        self.n_samples, self.n_features = X.shape
        #计算类别的先验概率
        self.calPy(y)
        print('P(y)训练完毕!')
        #yset = np.unique(y)
        #Pypa = {}
        Pxypa = {}
        # 第一层是不同的分类
        for yi, yiCount in self.ySet.items():
            Pxypa[yi] = {}
            # 第二层是不同的特征，如果有父属性，就接着加一层父属性，如果没有父属性，则按实际情况来
            for xiIdx in range(self.n_features):
                allXiset = np.unique(X.iloc[:, xiIdx])
                # 没有父属性的
                if xiIdx not in self.f_relationship.keys():
                    Xiarr = X.iloc[np.nonzero(np.array(y==yi))[0], xiIdx] # .flatten()
                    if columnsMark[xiIdx] == 0:
                        ## 保存离散特征的条件概率
                        Pxypa[yi][xiIdx] = self.__categorytrain(Xiarr, allXiset)
                    else:
                        ## 保存连续特征的条件概率
                        Pxypa[yi][xiIdx] = self.__continuoustrain(Xiarr)
                    continue
                
                # 第三层是有父属性的，值为父属性的各类值
                Pxypa[yi][xiIdx] = {}
                paIdx = self.f_relationship[xiIdx]
                paset = np.unique(X.iloc[:, paIdx])
                for pai in paset:
                    xi_pai_idx = np.nonzero(np.array((X.iloc[:,paIdx]==pai)&(y==yi)))[0]
                    Xiarr = X.iloc[xi_pai_idx, xiIdx]  #.flatten()
                    if columnsMark[xiIdx] == 0:
                        ## 保存离散特征的条件概率
                        Pxypa[yi][xiIdx][pai] = self.__categorytrain(Xiarr, allXiset)
                    else:
                        ## 保存连续特征的条件概率
                        Pxypa[yi][xiIdx][pai] = self.__continuoustrain(Xiarr)
        print('P(x|y,pa)训练完毕!')
        self.xyProba = Pxypa
        self.trainSet = X
        self.trainLabel = y
        self.columnsMark = columnsMark        
        return 

    # 计算离散特征的条件概率
    def __categorytrain(self, Xarr, xiset):
        pxypa = {}
        for xivalue in xiset:
            pxypa[xivalue] = {}
            pxypa[xivalue]['count'] = sum(Xarr==xivalue) + self.ls
            pxypa[xivalue]['ratio'] = self.classifyProba(xivalue, Xarr, len(xiset))
        return pxypa
    
    # 计算连续特征的均值和标准差
    def __continuoustrain(self, Xarr):
        pxypa = (Xarr.mean(), Xarr.std())
        return pxypa
        
    # 预测
    def tanpredict(self, X):
        n_samples, n_features = X.shape
        proba = np.zeros((n_samples, len(self.yProba)))
        for i in range(n_samples):
            for idx, (yi, Xidict) in enumerate(self.xyProba.items()):
                probaValue = 1.
                probaValue *= self.yProba[yi]
                for xiIdx, valuedict in Xidict.items():
                    xi = X.iloc[i, xiIdx]
                    ## 值不是字典，说明是连续变量
                    if not isinstance(valuedict, dict):
                        miu = valuedict[0]; sigma = valuedict[1] + 1.0e-5
                        Pxypa = np.exp(-(xi-miu)**2/(2*sigma**2))/(np.power(2*np.pi, 0.5)*sigma) + 1.0e-5
                    ## 第三层不是字典，说明特征没有依赖属性
                    elif not isinstance(list(list(valuedict.values())[0].values())[0], dict):
                        Pxypa = valuedict[xi]['ratio']
                    ## 第三层是字典，说明有依赖属性
                    else:
                        pai = X.iloc[i, self.f_relationship[xiIdx]]
                        Pxypa = valuedict[pai][xi]['ratio']
                    probaValue *= Pxypa
                proba[i, idx] = probaValue
        return proba
    
    
    # 取对数预测
    def tanpredictLog(self, X):
        n_samples, n_features = X.shape
        proba_log = np.zeros((n_samples, len(self.yProba)))
        for i in range(n_samples):
            for idx, (yi, Xidict) in enumerate(self.xyProba.items()):
                probaValueLog = 0.
                probaValueLog += np.log(self.yProba[yi])
                for xiIdx, valuedict in Xidict.items():
                    xi = X.iloc[i, xiIdx]
                    ## 值不是字典，说明是连续变量
                    if not isinstance(valuedict, dict):
                        miu = valuedict[0]; sigma = valuedict[1] + 1.0e-5
                        Pxypa = np.exp(-(xi-miu)**2/(2*sigma**2))/(np.power(2*np.pi, 0.5)*sigma) + 1.0e-5
                    ## 第三层不是字典，说明特征没有依赖属性
                    elif not isinstance(list(list(valuedict.values())[0].values())[0], dict):
                        Pxypa = valuedict[xi]['ratio']
                    ## 第三层是字典，说明有依赖属性
                    else:
                        pai = X.iloc[i, self.f_relationship[xiIdx]]
                        Pxypa = valuedict[pai][xi]['ratio']
                    probaValueLog += np.log(Pxypa)
                proba_log[i, idx] = probaValueLog
        return proba_log

In [ ]:
# (最小/最大)带权生成树相关的算法类(minimun/maxinum spanning tree)
class MST(object):
    
    def __init__(self, tree, method):
        """
        初始化带权生成树，可选择树类型、求解方法等参数

        Parameters
        ----------
        tree : str
            树的类型，包括max, min选择.
        method : str
            求解方法，包括prim,kruskal算法.

        """
        self.tree = tree.lower()
        self.method = method.lower()
        self.__reverse_flag = False if self.tree=='min' else True
    
    
    
    # 主体训练函数
    def fit_transform(self, points:list, edge:dict):
        if self.method == 'kruskal':
            edge_effective = self.__fit_kruskal(points, edge)
        else:
            pass
        return edge_effective
    
    
    
    # 克鲁斯卡尔算法(Kruskal算法)，从边出发
    def __fit_kruskal(self, points:list, edge:dict):
        # 点的下标，用来判断是否形成环路
        pointsmark = np.zeros(len(points))
        # 需要保留的边
        edgeLeft = []
        # 将边正序或者倒序排列
        edgeList = sorted(edge.items(), key=lambda x: x[1], reverse=self.__reverse_flag)
        # 判断是否形成环路：当边的两个点下标都为1时，则形成了环路
        for eg in edgeList:
            (p1, p2), w = eg
            if (pointsmark[points.index(p1)] == 1) and (pointsmark[points.index(p2)] == 1):
                continue
            else:
                pointsmark[points.index(p1)] = 1
                pointsmark[points.index(p2)] = 1
                edgeLeft.append(eg)        
        return edgeLeft
    
    
    